In [9]:
# imports
from transformers import AutoModelForSequenceClassification, AutoTokenizer


In [10]:
# Define your label mappings
id2label = {0: "None", 1: "User", 2: "Root"}
label2id = {"None": 0, "User": 1, "Root": 2}

# Load SecureBERT using AutoModelForSequenceClassification
model_name = "ehsanaghaei/SecureBERT"

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Load model for sequence classification
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=3,  # None, User, Root
    id2label=id2label,
    label2id=label2id
)

print(f"Model architecture: {type(model)}")
print(f"Number of labels: {model.num_labels}")
print(f"Label mappings: {model.config.id2label}")

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at ehsanaghaei/SecureBERT and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model architecture: <class 'transformers.models.roberta.modeling_roberta.RobertaForSequenceClassification'>
Number of labels: 3
Label mappings: {0: 'None', 1: 'User', 2: 'Root'}


## Training ##

In [ ]:

# 1. Import everything
from transformers import (
    AutoModelForSequenceClassification, 
    AutoTokenizer,
    TrainingArguments,
    Trainer,
    DataCollatorWithPadding,
    EarlyStoppingCallback
)
from peft import LoraConfig, get_peft_model
from datasets import Dataset
import evaluate
import numpy as np
import pandas as pd

# 2. Load model and tokenizer
model_name = "ehsanaghaei/SecureBERT"
id2label = {0: "None", 1: "User", 2: "Root"}
label2id = {"None": 0, "User": 1, "Root": 2}

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=3,
    id2label=id2label,
    label2id=label2id
)

# 3. Apply PEFT (LoRA) to optimize the training efficiency (since such small dataset)
lora_config = LoraConfig(
    task_type="SEQ_CLS",
    r=8,                    # Low rank for small dataset
    lora_alpha=16,          
    lora_dropout=0.0,       # Higher dropout for regularization
    target_modules=["query", "value", "key"],  # Target attention layers
    bias="none",             # Don't train bias terms
)

model = get_peft_model(model, lora_config)
print("PEFT Model Summary:")
model.print_trainable_parameters()

# 4. Load and prepare dataset 
df = pd.read_csv("../datasets/postcondition-dataset-finished.tsv", 
                    sep='\t', 
                    encoding='utf-8-sig',
                    keep_default_na=False,  
                    na_values=[''])

df.columns = df.columns.str.strip()
df = df.dropna(subset=['DESCRIPTION', 'CVSS', 'POSTCONDITION'])
df['POSTCONDITION'] = df['POSTCONDITION'].str.strip()
df['text'] = "Description: " + df['DESCRIPTION'] + " CVSS: " + df['CVSS']

dataset_dict = {
    "text": df['text'].tolist(),
    "label": df['POSTCONDITION'].tolist()
}
dataset = Dataset.from_dict(dataset_dict)

def format_labels(example):
    example["label"] = label2id[example["label"]]
    return example
dataset = dataset.map(format_labels)

def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True, max_length=512)
tokenized_dataset = dataset.map(tokenize_function, batched=True)

train_test_split = tokenized_dataset.train_test_split(test_size=0.2, seed=42)
train_dataset = train_test_split["train"]
eval_dataset = train_test_split["test"]

# 5. Setup evaluation
accuracy = evaluate.load("accuracy")
f1_metric = evaluate.load("f1")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    acc = accuracy.compute(predictions=predictions, references=labels)
    f1 = f1_metric.compute(predictions=predictions, references=labels, average="weighted")
    return {"accuracy": acc["accuracy"], "f1": f1["f1"]}

# 6. COMBINED: PEFT + Small Dataset Optimized Training Arguments
training_args = TrainingArguments(
    output_dir="./securebert-privilege-classifier",
    
    # Learning rate adjustments for small dataset
    learning_rate=1e-3,           
    weight_decay=0.0,           # Regularization
    
    # Batch size adjustments
    per_device_train_batch_size=1,  # Very small batch size
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=1,  
    
    # Epoch and strategy adjustments
    num_train_epochs=100,           # More epochs for small dataset
    eval_strategy="epoch",        # Evaluate every epoch
    save_strategy="epoch",        # Save every epoch
    load_best_model_at_end=True,  # Load best model
    metric_for_best_model="f1",
    greater_is_better=True,
    
    # Early stopping and regularization
    warmup_ratio=0.1,             # 10% warmup
    max_grad_norm=1.0,            # Gradient clipping
    

    seed=42,
    data_seed=42,
    
    # Performance optimizations
    fp16=True,                    # Mixed precision if GPU available
    ddp_find_unused_parameters=False,
    
    # PEFT specific
    remove_unused_columns=True,
)

# 7. Create Trainer with Early Stopping
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=10)]
)

# 8. Train
print("Starting training with PEFT + optimized args...")
trainer.train()

# 9. Save
print("Saving model...")
trainer.save_model("./securebert-privilege-classifier-final")
tokenizer.save_pretrained("./securebert-privilege-classifier-final")

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at ehsanaghaei/SecureBERT and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


PEFT Model Summary:
trainable params: 1,035,267 || all params: 125,683,206 || trainable%: 0.8237


Map:   0%|          | 0/33 [00:00<?, ? examples/s]

Map:   0%|          | 0/33 [00:00<?, ? examples/s]

/tmp/ipykernel_99131/2631405035.py:123: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForSequenceClassification`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Starting training with PEFT + optimized args...


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,No log,1.109933,0.142857,0.035714
2,No log,1.145787,0.285714,0.142857
3,No log,1.224609,0.428571,0.333333
4,No log,1.366769,0.428571,0.333333
5,No log,3.878258,0.285714,0.163265
6,No log,4.079762,0.428571,0.261905
7,No log,4.180416,0.428571,0.261905
8,No log,4.331934,0.571429,0.571429
9,No log,6.467332,0.428571,0.261905
10,No log,5.503678,0.571429,0.528571


Saving model...


('./securebert-privilege-classifier-final/tokenizer_config.json',
 './securebert-privilege-classifier-final/special_tokens_map.json',
 './securebert-privilege-classifier-final/vocab.json',
 './securebert-privilege-classifier-final/merges.txt',
 './securebert-privilege-classifier-final/added_tokens.json',
 './securebert-privilege-classifier-final/tokenizer.json')

## Model Testing and Results ##

In [21]:
import pandas as pd
import torch
from sklearn.metrics import f1_score, classification_report

def test_on_held_out_set(model, tokenizer, test_dataset, output_file="securebert_test_results.tsv"):
    """Test the trained model on the held-out test set and save results"""
    
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    model.eval()
    
    results = []
    
    print(f"Testing model on {len(test_dataset)} samples...")
    
    for i, sample in enumerate(test_dataset):
        # Reconstruct the original text
        text = tokenizer.decode(sample['input_ids'], skip_special_tokens=True)
        
        # Get prediction
        inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=512)
        inputs = {k: v.to(device) for k, v in inputs.items()}
        
        with torch.no_grad():
            outputs = model(**inputs)
            logits = outputs.logits
            predictions = torch.nn.functional.softmax(logits, dim=-1)
            predicted_class = torch.argmax(predictions, dim=1).item()
            confidence = predictions[0][predicted_class].item()
        
        # Convert labels back to text
        true_label = model.config.id2label[sample['label']]
        predicted_label = model.config.id2label[predicted_class]
        
        # Get all probabilities
        all_probs = {
            model.config.id2label[i]: predictions[0][i].item() 
            for i in range(len(model.config.id2label))
        }
        
        results.append({
            'text': text,
            'true_label': true_label,
            'predicted_label': predicted_label,
            'true_label_id': sample['label'],
            'predicted_label_id': predicted_class,
            'confidence': confidence,
            'correct': true_label == predicted_label,
            'prob_none': all_probs['None'],
            'prob_user': all_probs['User'],
            'prob_root': all_probs['Root']
        })
        
        if (i + 1) % 10 == 0:
            print(f"Processed {i + 1}/{len(test_dataset)} test samples")
    
    # Save results as TSV
    df_results = pd.DataFrame(results)
    df_results.to_csv(output_file, sep='\t', index=False, encoding='utf-8')
    
    # Calculate metrics
    accuracy = df_results['correct'].mean()
    total_samples = len(df_results)
    correct_predictions = df_results['correct'].sum()
    
    # Calculate F1 scores
    y_true = df_results['true_label_id'].tolist()
    y_pred = df_results['predicted_label_id'].tolist()
    
    f1_macro = f1_score(y_true, y_pred, average='macro')
    f1_weighted = f1_score(y_true, y_pred, average='weighted')
    f1_micro = f1_score(y_true, y_pred, average='micro')
    
    print(f"\n=== TEST SET RESULTS ===")
    print(f"Total samples: {total_samples}")
    print(f"Correct predictions: {correct_predictions}")
    print(f"Accuracy: {accuracy:.4f} ({accuracy*100:.2f}%)")
    print(f"F1 Score (Macro): {f1_macro:.4f}")
    print(f"F1 Score (Weighted): {f1_weighted:.4f}")
    print(f"F1 Score (Micro): {f1_micro:.4f}")
    
    # Per-class breakdown with F1 scores
    print(f"\n=== PER-CLASS RESULTS ===")
    class_f1_scores = f1_score(y_true, y_pred, average=None)
    for i, label in enumerate(['None', 'User', 'Root']):
        subset = df_results[df_results['true_label'] == label]
        if len(subset) > 0:
            class_accuracy = subset['correct'].mean()
            class_f1 = class_f1_scores[i]
            print(f"{label}: {len(subset)} samples, {class_accuracy:.4f} accuracy ({class_accuracy*100:.1f}%), F1: {class_f1:.4f}")
            
            # Show what this class was predicted as
            pred_counts = subset['predicted_label'].value_counts()
            for pred_label, count in pred_counts.items():
                percentage = (count / len(subset)) * 100
                print(f"  -> Predicted as {pred_label}: {count} ({percentage:.1f}%)")
    
    # Detailed classification report
    print(f"\n=== DETAILED CLASSIFICATION REPORT ===")
    target_names = ['None', 'User', 'Root']
    print(classification_report(y_true, y_pred, target_names=target_names, digits=4))
    
    # Confidence analysis
    print(f"\n=== CONFIDENCE ANALYSIS ===")
    high_conf = df_results[df_results['confidence'] >= 0.9]
    med_conf = df_results[(df_results['confidence'] >= 0.7) & (df_results['confidence'] < 0.9)]
    low_conf = df_results[df_results['confidence'] < 0.7]
    
    if len(high_conf) > 0:
        high_f1 = f1_score(high_conf['true_label_id'], high_conf['predicted_label_id'], average='weighted')
        print(f"High confidence (≥90%): {len(high_conf)} samples, {high_conf['correct'].mean():.4f} accuracy, F1: {high_f1:.4f}")
    
    if len(med_conf) > 0:
        med_f1 = f1_score(med_conf['true_label_id'], med_conf['predicted_label_id'], average='weighted')
        print(f"Medium confidence (70-90%): {len(med_conf)} samples, {med_conf['correct'].mean():.4f} accuracy, F1: {med_f1:.4f}")
    
    if len(low_conf) > 0:
        low_f1 = f1_score(low_conf['true_label_id'], low_conf['predicted_label_id'], average='weighted')
        print(f"Low confidence (<70%): {len(low_conf)} samples, {low_conf['correct'].mean():.4f} accuracy, F1: {low_f1:.4f}")
    
    print(f"\nResults saved to: {output_file}")
    
    return df_results

# Test the model on the held-out test set
print("Testing trained model on held-out test set...")
test_results = test_on_held_out_set(model, tokenizer, eval_dataset, "securebert_test_results.tsv")

Testing trained model on held-out test set...
Testing model on 7 samples...

=== TEST SET RESULTS ===
Total samples: 7
Correct predictions: 5
Accuracy: 0.7143 (71.43%)
F1 Score (Macro): 0.7222
F1 Score (Weighted): 0.7381
F1 Score (Micro): 0.7143

=== PER-CLASS RESULTS ===
None: 1 samples, 1.0000 accuracy (100.0%), F1: 0.5000
  -> Predicted as None: 1 (100.0%)
User: 4 samples, 0.5000 accuracy (50.0%), F1: 0.6667
  -> Predicted as None: 2 (50.0%)
  -> Predicted as User: 2 (50.0%)
Root: 2 samples, 1.0000 accuracy (100.0%), F1: 1.0000
  -> Predicted as Root: 2 (100.0%)

=== DETAILED CLASSIFICATION REPORT ===
              precision    recall  f1-score   support

        None     0.3333    1.0000    0.5000         1
        User     1.0000    0.5000    0.6667         4
        Root     1.0000    1.0000    1.0000         2

    accuracy                         0.7143         7
   macro avg     0.7778    0.8333    0.7222         7
weighted avg     0.9048    0.7143    0.7381         7


=== CO

## Using the model ## 

In [ ]:
def predict_privilege(description, cvss_vector, model, tokenizer):
    """Predict privilege level using the fine-tuned model"""
    # Prepare input
    text = f"Description: {description} CVSS: {cvss_vector}"
    
    # Tokenize
    inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=512)
    
    # Move to device
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    inputs = {k: v.to(device) for k, v in inputs.items()}
    
    # Predict
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        predictions = torch.nn.functional.softmax(logits, dim=-1)
        predicted_class = torch.argmax(predictions, dim=1).item()
        confidence = predictions[0][predicted_class].item()
    
    return {
        "predicted_privilege": model.config.id2label[predicted_class],
        "confidence": confidence,
        "all_probabilities": {
            model.config.id2label[i]: predictions[0][i].item() 
            for i in range(len(model.config.id2label))
        }
    }

# Example
description = "Buffer overflow allowing arbitrary code execution"
cvss_vector = "CVSS:3.1/AV:N/AC:L/PR:N/UI:N/S:U/C:H/I:H/A:H"

result = predict_privilege(description, cvss_vector, model, tokenizer)
print(f"Predicted privilege: {result['predicted_privilege']}")
print(f"Confidence: {result['confidence']:.2%}")
print(f"All probabilities: {result['all_probabilities']}")